In [2]:
from dotenv import load_dotenv
import os
import lyricsgenius
import yaml
import tomllib

In [13]:
# Get credentials for the api calls from the .env file
# Niko has the credentials if you need them
load_dotenv()

root = "../../.."

# Get config entries
with open(f"{root}/config.toml", "rb") as f:
    config = tomllib.load(f)

In [4]:
genius = lyricsgenius.Genius()

In [4]:
# Scraping of all existing secondary tags on genius 
# has been done in src/notebooks/scrape-secondary-tags.ipynb
main_tags = ["country", "pop", "r-b", "rap", "rock"]
with open(root + config["data_dir"] + "/secondary_genres.yaml", "r") as f:
    secondary_tags = yaml.safe_load(f)

main_tags, secondary_tags

(['country', 'pop', 'r-b', 'rap', 'rock'],
 ['abstract-rap',
  'a-cappella',
  'acid-house',
  'acid-jazz',
  'acid-techno',
  'acoustic',
  'adult-alternative',
  'adult-contemporary',
  'afro-arab',
  'afrobeat',
  'afrobeats',
  'afro-hip-hop',
  'afro-house',
  'afro-jazz',
  'afrosoul',
  'afroswing',
  'afro-trap',
  'albanian-folk',
  'album-oriented-rock-aor',
  'algerian-rai',
  'algerian-rap',
  'alternative',
  'alternative-country',
  'alternative-dance',
  'alternative-metal',
  'alternative-pop',
  'alternative-rap',
  'alternative-r-b',
  'alternative-rock',
  'amapiano',
  'ambient',
  'ambient-dub',
  'ambient-pop',
  'americana',
  'american-folk',
  'american-underground',
  'anarcho-punk',
  'anime-lo-fi',
  'anime-rap',
  'anthem',
  'anti-folk',
  'arabic-diss',
  'arabic-instrumental',
  'arabic-pop',
  'arabic-rap',
  'arabic-rock',
  'arrocha',
  'art-pop',
  'art-punk',
  'art-rock',
  'ashiq-music',
  'atmospheric-black-metal',
  'atmospheric-sludge-metal',
 

In [7]:
# Try to find all songs for a specified genre
abstract_rap_songs = genius.tag("abstract-rap")["hits"]

In [8]:
# These are appearently not all songs yet
len(abstract_rap_songs)

20

In [9]:
feel_good_inc = abstract_rap_songs[2]
feel_good_inc

{'url': 'https://genius.com/Gorillaz-feel-good-inc-lyrics',
 'title_with_artists': 'Feel Good Inc. by Gorillaz (Ft. De La Soul)',
 'title': 'Feel Good Inc.',
 'artists': ['Gorillaz'],
 'featured_artists': ['De La Soul']}

In [10]:
# Print the lyrics for a specific song
print(genius.lyrics(song_url=feel_good_inc["url"]))

348 ContributorsTranslationsPolskiItalianoNederlandsFeel Good Inc. Lyrics[Intro: Maseo & 2-D]
Hahahahahahahahahahahahahahahahaha
Feel good
Sha, sha-ba-da, sha-ba-da-ca; feel good
Sha, sha-ba-da, sha-ba-da-ca; feel good
Sha, sha-ba-da, sha-ba-da-ca; feel good
Sha, sha-ba-da, sha-ba-da-ca; feel good
Sha, sha-ba-da, sha-ba-da-ca; feel good
Sha, sha-ba-da, sha-ba-da-ca; feel good
(Change, change, change, change)
Sha, sha-ba-da, sha-ba-da-ca; feel good
(Change, change, change, change)
Sha, sha-ba-da, sha-ba-da-ca; feel good

[Verse 1: 2-D]
City's breaking down on a camel's back
They just have to go, 'cause they don't know wack
So while you fill the streets, it's appealing to see
And you won't get out the county 'cause you're damned and free
You got a new horizon, its ephemeral style
A melancholy town where we never smile
And all I wanna hear is the message beep
My dreams, they've got to kiss me 'cause I don't get to sleep, no
(Beep)

[Chorus: 2-D]
Windmill, windmill for the land
Turn foreve

In [ ]:
abstract_rap_songs[:2]

[{'url': 'https://genius.com/Childish-gambino-iv-sweatpants-lyrics',
  'title_with_artists': 'IV. Sweatpants by Childish Gambino',
  'title': 'IV. Sweatpants',
  'artists': ['Childish Gambino'],
  'featured_artists': []},
 {'url': 'https://genius.com/Tyler-the-creator-yonkers-lyrics',
  'title_with_artists': 'Yonkers by Tyler, The Creator',
  'title': 'Yonkers',
  'artists': ['Tyler, The Creator'],
  'featured_artists': []},
 {'url': 'https://genius.com/Gorillaz-feel-good-inc-lyrics',
  'title_with_artists': 'Feel Good Inc. by Gorillaz (Ft. De La Soul)',
  'title': 'Feel Good Inc.',
  'artists': ['Gorillaz'],
  'featured_artists': ['De La Soul']}]

In [ ]:
# The page feature can be used to find 20 more songs
genius.tag("abstract-rap", page=2)["hits"][:2]

[{'url': 'https://genius.com/Chance-the-rapper-acid-rain-lyrics',
  'title_with_artists': 'Acid Rain by Chance the Rapper',
  'title': 'Acid Rain',
  'artists': ['Chance the Rapper'],
  'featured_artists': []},
 {'url': 'https://genius.com/A-ap-rocky-phoenix-lyrics',
  'title_with_artists': 'Phoenix by A$AP Rocky',
  'title': 'Phoenix',
  'artists': ['A$AP Rocky'],
  'featured_artists': []}]

In [17]:
# Download songs for every genre
for genre in config["genres"]:

    # Genius does not return more than 50 pages
    n_pages = int(config["n_songs"][genre] / 20)
    n_pages = min(n_pages, 50)

    # Iterate through pages of songs in genius
    titles = []
    lyrics = []
    all_songs = []
    for i in range(1, n_pages + 1):

        songs_on_page = None
        while not songs_on_page:
            try:
                songs_on_page = genius.tag(genre, page=i)["hits"]
                if not songs_on_page:
                    raise ValueError(f"Page {i} for the {genre} genre is empty")
            except Exception as e:
                print(f"❌ Failed to load page {i} for the {genre} genre")

        for song in songs_on_page:
            title = song["title_with_artists"]

            # A Page "Behind The Lyrics" appeared in the country songs
            # This is not a song and is filtered out
            if title.startswith("Behind The Lyrics"):
                continue

            all_songs.append(song)
            print(f"Saved '{title}'")

    with open(f"{root}/{config['multilabel_data_dir']}/songs/{genre}.yml", "w") as f:
        yaml.safe_dump(all_songs, f)
    print(f"Succsefffully saved {genre} songs")

KeyboardInterrupt: 

In [ ]:
# Clear the lyrics for a given set of genres
genres_to_clear = ["r-b", "jazz", "experimental", "electronic"]
for genre in genres_to_clear:
    lyric_dir = f"{root}/{config['multilabel_data_dir']}/lyrics/{genre}"
    for lyric in os.listdir(lyric_dir):
        os.remove(f"{lyric_dir}/{lyric}")

In [11]:
# Download lyrics for all the songs

for genre in config["genres"]:

    lyric_dir = f"{root}/{config['multilabel_data_dir']}/lyrics/{genre}"

    # Create an output folder
    if not os.path.exists(lyric_dir):
        os.mkdir(f"{root}/{config['multilabel_data_dir']}/lyrics/{genre}")

    # Get the song list for one genre
    with open(f"{root}/{config['multilabel_data_dir']}/songs/{genre}.yml", "r") as f:
        songs = yaml.safe_load(f)

    # The API call for lyrics often times out
    # We repeat it every time it has timed out
    error = True
    while error:
        error = False 

        try:
            # Check the already saved songs
            saved_songs = os.listdir(lyric_dir)
            n_saved_songs = len(saved_songs)

            # Download the song lyrics
            for i, song in enumerate(songs[n_saved_songs:]):
                title: str = song["title_with_artists"]
                url: str = song["url"]

                # The title sometimes has invalid chars for a file name
                # Remove these here
                scary_chars = "\"/?*|"
                for char in scary_chars:
                    title = title.replace(char, "")
                if len(title) > 130:
                    title = title[:130]

                lyrics: str = genius.lyrics(song_url=url)

                # The downlaoded lyrics have a header in the first line and
                # a number + "Embed" on the last line
                # These need to be removed
                lyrics = "\n".join(lyrics.splitlines()[1:])[:-5]
                
                # Skip emtpy lyrics
                if len(lyrics) == 0:
                    continue

                # Remove "Embed" + number from the end
                if lyrics[-1] == 'K':
                    lyrics = lyrics[:-3]
                while lyrics[-1].isdigit():
                    lyrics = lyrics[:-1]

                with open(f"{root}/{config['multilabel_data_dir']}/lyrics/{genre}/{(n_saved_songs + i):04d}_{title}.txt", "w", encoding="utf-8") as f:
                    f.write(lyrics)
                print(f"✅ Saved {genre} {n_saved_songs + i}: {title}")
        except Exception as e:
            print(f"❌ {e}")
            error = True

✅ Saved blues 989: Makes Me Wanna Pray by Christina Aguilera (Ft. Steve Winwood)
✅ Saved blues 990: Nobody Loves Me But My Mother by B.B. King
✅ Saved blues 991: The Itch by Keb' Mo'
✅ Saved blues 992: It's All Over But the Crying by Garbage
✅ Saved blues 993: A Sweet Little Bullet from a Pretty Blue Gun by Tom Waits
✅ Saved blues 994: Misery by P!nk (Ft. Steven Tyler)
✅ Saved blues 995: He’s My Thing (EP Version) by Babes In Toyland
✅ Saved blues 996: Hurt Like Mine by The Black Keys
✅ Saved blues 997: Sleeper 1972 by Manchester Orchestra
✅ Saved blues 998: Hesitation Blues by Hot Tuna
✅ Saved jazz 979: Frank's Wild Years by Tom Waits
✅ Saved jazz 980: Lover Man by Billie Holiday
✅ Saved jazz 981: Plaza Avenue by Ardhito Pramono
✅ Saved jazz 982: Slowly Growing Deaf (Sleep: Part I) by Mr. Bungle
✅ Saved jazz 983: Step Right Up by Tom Waits
✅ Saved jazz 984: Swing Low Sweet Chariot by Louis Armstrong
✅ Saved jazz 985: Fear by Sade
✅ Saved jazz 986: A Night Like This by Caro Emerald
✅ S